In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text 
from sklearn.linear_model import Logimport TfidfVectorizeristicRegression
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('../data/IMDb_50K.csv')

In [ ]:
print(df.head())
print(df['sentiment'].value_counts())

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r"[^a-zA-Z']", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df['review_clean'] = df['review'].apply(clean_text)

In [ ]:
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review_clean'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

In [ ]:
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))